In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors  # Correct import for colors module
from scipy.ndimage import gaussian_filter
import tifffile as tiff
import os
from PIL import Image
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.models import LinearColorMapper, ColorBar, HoverTool, CustomJS, ColumnDataSource, CheckboxGroup, Slider
from bokeh.layouts import column
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral3  # Palette with three colors
import ipywidgets as widgets
import sys

# Assuming the .pyd file is located at the given path
sys.path.append(r"C:\Users\jediati\Desktop\JEDIATI\builds\test_GradIntegrator\bin\Release")
import msc_py

# Enable Bokeh output in Jupyter
output_notebook()

DEBUG = True

## read in the 2 channels

In [ ]:
I1 = r"C:\Users\jediati\Desktop\JEDIATI\data\ARPA-H\Area1_470.tif"
I2 = r"C:\Users\jediati\Desktop\JEDIATI\data\ARPA-H\Area1_528.tif"
DEBUG_OUTPUT_GRAY_FILE = r"C:\Users\jediati\Desktop\JEDIATI\data\ARPA-H\Area1_COMB.tif"

image = Image.open(I1)
image2 = Image.open(I2)
# Convert to grayscale
image_gray1 = image#.convert('L')  # 'L' mode is for grayscale
image_gray2 = image2#.convert('L')  # 'L' mode is for grayscale
a1 = np.array(image_gray1, dtype=np.float32)
a2 = np.array(image_gray2, dtype=np.float32)


### do channels to rgba

In [ ]:
low_in_470, high_in_470 = [0.0, 64774.0]  # precomputed
low_in_580, high_in_580 = [0.0, 65517.0]  # precomputed
cI = 0.8
nI = 0.8
cG = 0.7
nG = 0.8

# ///////////////////////////////////////////////////////////////////////
def gammaScale(image, gamma, low_in, high_in):
    new_img = np.power(np.divide(np.subtract(image, low_in), high_in - low_in), gamma)
    return new_img

# ///////////////////////////////////////////////////////////////////////
def imageScale(image, scale):
    new_img = np.multiply(image, scale)
    return new_img

tile_red = imageScale(a1, nI)
tile_red = gammaScale(a1, nG, low_in_470, high_in_470)

tile_blue = imageScale(a2, cI)
tile_blue = gammaScale(a2, cG, low_in_580, high_in_580)



def ibMGColorReMap(ired, iblue):
    B_Beta_E = 0.7000
    B_Beta_H = 0.5000
    c = 0.0821
    G_Beta_E = 0.9000
    G_Beta_H = 1.0
    k = 1.0894
    R_Beta_E = 0.0200
    R_Beta_H = 0.8600

    I_r = ((np.exp(-R_Beta_H * ired * 2.5) - c) * k) * ((np.exp(-R_Beta_E * iblue * 2.5) - c) * k)
    I_g = ((np.exp(-G_Beta_H * ired * 2.5) - c) * k) * ((np.exp(-G_Beta_E * iblue * 2.5) - c) * k)
    I_b = ((np.exp(-B_Beta_H * ired * 2.5) - c) * k) * ((np.exp(-B_Beta_E * iblue * 2.5) - c) * k)

    rgb = np.squeeze(np.stack((I_r, I_g, I_b), axis=-1))
    rgb *= 255
    rgb = rgb.astype("uint8")
    return rgb

rgb = ibMGColorReMap(tile_red, tile_blue)


In [ ]:
X, Y, _ = rgb.shape
# Step 1: Convert RGB to RGBA by adding an alpha channel
rgba_image = np.dstack((rgb, 255 * np.ones((X, Y), dtype=np.uint8)))
rgba_flat = np.zeros((X, Y), dtype=np.uint32)
view = rgba_flat.view(dtype=np.uint8).reshape((X, Y, 4))
view[:, :, :] = rgba_image

## make a grayscale function on which to compute topology

In [ ]:
print(np.max(a1), np.max(a2))
ba1 = gaussian_filter(np.sqrt(a1), sigma=2).astype(np.float32)
ba2 = gaussian_filter(np.sqrt(a2), sigma=2).astype(np.float32)
blurred_array = (ba2+ba1)*0.5

In [ ]:
# Visualize the array
plt.imshow(blurred_array, cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
if DEBUG:
    file_root = os.path.splitext(DEBUG_OUTPUT_GRAY_FILE)[0]
    outname = "{}_{}x{}.raw".format(file_root, blurred_array.shape[1], blurred_array.shape[0])
    print("writing raw:", outname)
    blurred_array.tofile(outname)

# Do the topology computation

## create a c++ side msc structure that will be filled in

In [ ]:
id = msc_py.MakeMSCInstance()

## does the heavy lifting - computes discrete gradient, and a MSC + hierarchy up to 20% simplification.

In [ ]:
%%time 
msc_py.ComputeMSC(id, blurred_array)

## example query the "mountains" of the segmentation - also first run caches the base segmentation so future runs are faster

In [ ]:
%%time 
# get the basins at persistece 50
msc_py.SetMSCPersistence(id, 0.00)
id_array = msc_py.GetAsc2Manifolds(id)

## Interactive exploration of the image and segmentation

In [ ]:
%%time

# Assuming 'nodes' is your DataFrame with "x", "y", "dim", and "value" columns

# Create a color map for 'dim' values: 0 -> Blue, 1 -> Green, 2 -> Red
color_map = factor_cmap('dim_str', palette=['dodgerblue', 'lawngreen', 'orangered'], factors=['0', '1', '2'])


# Recreate the color map (matplotlib colormap)
num_colors = 63
random_colors = np.random.rand(num_colors, 3)
cmap = mcolors.ListedColormap(random_colors)

# Generate colormap as a Bokeh-compatible palette (RGB hex values)
palette = [cmap(i / num_colors) for i in range(num_colors)]
palette = [mcolors.rgb2hex(c[:3]) for c in palette]  # Convert to hex colors

figwidth = 900
# Create a Bokeh figure
p = figure(
    x_range=(0, blurred_array.shape[1]), 
    y_range=(0, blurred_array.shape[0]), 
    width=figwidth,
    height=int(figwidth*(blurred_array.shape[0]/blurred_array.shape[1])),
    tools="pan,wheel_zoom,box_zoom,reset,save",
    title="Interactive Image with Bokeh",
)


# Display the image

ba2_flipped = np.flipud(rgba_flat)
image_renderer = p.image_rgba(image=[ba2_flipped], x=0, y=0, dw=blurred_array.shape[1], dh=blurred_array.shape[0])
# Map the id_array2 values to the colormap with LinearColorMapper
color_mapper = LinearColorMapper(palette=palette, low=0, high=num_colors-1)


# id_flipped = np.flipud(id_array2)
# mask = ba2_flipped > 20
# id_flipped_masked = np.where(mask, id_flipped, np.nan)
def change_persistence(value):
    msc_py.SetMSCPersistence(id, value)
    id_array2 = msc_py.GetDsc2Manifolds(id)
    nodes = pd.DataFrame(msc_py.GetCriticalPoints(id))
    nodes['dim_str'] = nodes['dim'].astype(str)
    nodes['y_f'] = blurred_array.shape[0] - nodes['y']
    return np.flipud(id_array2), nodes

id_flipped_masked, nodes = change_persistence(1.0)
source = ColumnDataSource(nodes)
overlay_source = ColumnDataSource(data=dict(image=[id_flipped_masked % num_colors]))
# Add the colormapped overlay with alpha blending
overlay_renderer = p.image(image='image', x=0, y=0, dw=id_array.shape[1], dh=id_array.shape[0],
                           source=overlay_source, color_mapper=color_mapper, alpha=0.3)


# Add circles to the plot with colors based on 'dim'
p.circle(
    x='x', 
    y='y_f', 
    size=3, 
    color=color_map, 
    source=source, 
    legend_field='dim_str',
    fill_alpha=0.6
)
p.legend.click_policy = "hide"

# Hover tool for the image (to show pixel values)
hover_image = HoverTool(renderers=[p.renderers[0]])  # Target the image
hover_image.tooltips = [("Image Value", "@image{0.00}")]
hover_image.point_policy = "follow_mouse"

# Add a hover tool to display the "value" column
# Hover tool for the points (to show point values)
hover_points = HoverTool(renderers=[p.renderers[2]])  # Target the points
hover_points.tooltips = [("Point Value", "@value")]
hover_points.point_policy = "follow_mouse"


#p.add_tools(hover_image)
p.add_tools(hover_points)
# Show the plot
# Create a CheckboxGroup for showing/hiding the images
checkbox = CheckboxGroup(labels=["Show RGBA Image", "Show MSC Segmentation"], active=[0, 1])

# Add a callback to toggle visibility of the image renderers
checkbox_callback = CustomJS(args=dict(image_renderer=image_renderer, overlay_renderer=overlay_renderer), code="""
    // Toggle RGBA Image visibility
    image_renderer.visible = cb_obj.active.includes(0);
    
    // Toggle Overlay Image visibility
    overlay_renderer.visible = cb_obj.active.includes(1);
""")

checkbox.js_on_change('active', checkbox_callback)

# Layout the figure and checkbox
layout = column(checkbox, p)

# Show the plot
handle = show(layout, notebook_handle=True)

mymax = np.max(blurred_array) * 0.2
# Create the slider with a custom width
slider = widgets.FloatSlider(
    value=1.0, 
    min=0, 
    max=mymax, 
    step=0.1, 
    description='Persistence',
    layout=widgets.Layout(width='900px')  # Set the width to 900 pixels
)

# Define the function to be called when the slider value changes
def on_slider_change(change):
    persistence_value = change['new']
    id_flipped_masked, nodes = change_persistence(persistence_value)  # Update the persistence value
    overlay_source.data = dict(image=[id_flipped_masked % num_colors])  # Update the image data
    # Update the ColumnDataSource for the circles
    source.data = dict(
        x=nodes['x'],
        y_f=nodes['y_f'],
        dim_str=nodes['dim_str'],
        value=nodes['value']
    )
    push_notebook(handle=handle)  # Update the plot in the notebook
    

# Attach the function to the slider
slider.observe(on_slider_change, names='value')

# Display the slider in the notebook
widgets.VBox([slider])

In [ ]:
# # you can also programmatically update the plot above by ending the cell with push_notebook()
# persistence_value = 100
# id_flipped_masked = change_persistence(persistence_value)  # Call the function to update persistence
# overlay_source.data = dict(image=[id_flipped_masked % num_colors])  # Update the image data
# push_notebook()